In [ ]:
from keys import keys

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import json
import pandas as pd
import requests
from datetime import datetime, timedelta

# XE api
from xecd_rates_client import XecdClient


In [ ]:
class Create_DF:
    def __init__(self, cur_original, cur_new, xecd_client):
        self.dfs_cur = {
            'CAD':pd.read_csv('data/articles/CAD.csv'),
            'USD':pd.read_csv('data/articles/USD.csv'),
            'JPY':pd.read_csv('data/articles/JPY.csv'),
            'CNY':pd.read_csv('data/articles/CNY.csv'),
            'EUR':pd.read_csv('data/articles/EUR.csv'),
            'INR':pd.read_csv('data/articles/INR.csv'),
        }
        self.original = cur_original
        self.new = cur_new
        self.xecd = xecd_client
    
    
    def __get_week_before_date(self,curr_date):
        week_ago = curr_date - timedelta(days=6)
        return week_ago
        
        
    def __get_weekly_rate(self, date, fromCur, toCur):
        xecd = self.xecd
        sum_rates = 0
        week_ago = self.__get_week_before_date(date)
        rateResult = xecd.historic_rate_period(1,fromCur, toCur, str(week_ago),str(date))
        
        for rate in rateResult['to'][self.new]:
            sum_rates = sum_rates + rate['mid']
        
        sum_rates = sum_rates / 7
        return sum_rates
    
    
    def __get_diff_sentiments(self,x,df1,df2):
        arr_sent = []
        for i in range(len(x)):
            original_value = df1['score'][i]
            new_value = df2['score'][i]
            diff = original_value - new_value
            arr_sent.append(diff)
        return arr_sent
        
        
    def __arr_get_sentiment_over_rate(self, x, df1, df2):
        sent_over_rate = []
        for i in range(len(x)):
            diff = self.__get_diff_sentiments(x,df1,df2)
            rate = self.__get_weekly_rate(x[i], self.original, self.new)
            
            value = diff[i] / rate
            sent_over_rate.append(value)
        return sent_over_rate
        
        
    def same_day_publications(self,df):
        df['currency'] = self.original
        df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
        df = df.groupby(['currency', pd.Grouper(key='date', freq='W-MON')])['score'].sum().reset_index().sort_values('date')
        return df
    
    
    def get_relation(self, original, new):
        df_original = self.same_day_publications(self.dfs_cur[self.original])
        df_new = self.same_day_publications(self.dfs_cur[self.new])
        xchange = []
        
        if set(df_original['date']) > set(df_new['date']):
            x = sorted(set(df_original['date']))
        else:
            x = sorted(set(df_new['date']))
        
        sent_y = self.__arr_get_sentiment_over_rate(x,df_original,df_new)
        
        for i in range(len(x)):
            xchange.append(self.__get_weekly_rate(x[i],self.original,self.new))

            
        # Graphs
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=x, y=xchange, mode='lines',name='Exchange Rate',connectgaps=True),secondary_y=True)
        fig.add_trace(go.Scatter(x=x, y=sent_y, mode='lines',name='Sentiment Over Rate',connectgaps=True))
        fig.add_trace(go.Scatter(x=x, y=self.__get_diff_sentiments(x,df_original,df_new), mode='lines',name='Diff in Sentiment',connectgaps=True))
        
        fig.update_yaxes(title_text="Sentiment Value", secondary_y=False)
        fig.update_yaxes(title_text="Exchange Rate", secondary_y=True)
        fig.show()
        
        
    def analyze_currencies(self):
        self.get_relation(self.original,self.new)


## Input Parameter

In [ ]:
news_api_key = keys['news']
json_google_cloud = keys['json_google_cloud']
xecd_client = keys['xe']
rapid_api_host = keys['rapid_api_host']
rapid_api_key = keys['rapid_api_key']

In [ ]:
print("Enter the original currency: ")
cur_original = input()

print("Enter the new currency: ")
cur_return = input()

## Create Dataframe

In [ ]:
created_df = Create_DF(cur_original, cur_return, xecd_client)

In [ ]:
df = pd.read_csv('data/articles/USD.csv')
df1 = created_df.analyze_currencies()